In [1]:
import snappy, csv, pickle
with open("braid_words_pickle.txt", "rb") as braid_words: 
	words = pickle.load(braid_words)

In [2]:
def get_degrees(A): 
#     temp = str(A).replace(' 1', ' a^0')
#     temp = temp.replace(' a ', ' a^1 ')
    R.<a> = LaurentPolynomialRing(ZZ, 'a')
    # coeffs = re.findall(r'\d+', temp)
    # retval = []
    # for t in coeffs: 
    #     retval.append(eval(t))
    return R(A).exponents()

def get_Ordv_Alexander(A):
    ordv = 0
    alpha = get_degrees(A)
    for i in range(1,len(alpha)):
        diff = abs(alpha[i-1] - alpha[i])
        ordv = max(ordv, diff)
    return ordv

def get_Ordv_laurent(P):
    exp = P.exponents()
    retVal = 0
    for i in range(1, len(exp)):
        if abs(exp[i] - exp[i-1]) > retVal: 
            retVal = abs(exp[i] - exp[i-1])
    return retVal

def get_Ordv_HFK(K):
    L = snappy.Manifold(K).link()
    F = PolynomialRing(Integers(2), 'v')
    HFK = L.knot_floer_homology(complex=True)
    return get_Ordv_from_complex(HFK)

def get_Ordv_from_complex(HFK):
    F = PolynomialRing(Integers(2), 'v')
    diff = HFK['differentials'] #gens x gens \mapsto <gens>
    gens = HFK['generators'] #{i: (gr_A(i), gr_M(i))}
    diff_v = {(i,j):F(diff[i,j]*F('v')^(gens[i][0]-gens[j][0])) for i, j in diff if gens[i][1] == gens[j][1] + 1}
    mat_v = matrix(F, diff_v)
    M = mat_v.elementary_divisors()
    ord_v = 0
    for i in M: 
        P = F(i)
        if(str(i) == '0'): continue
        deg = P.exponents()[-1]
        ord_v = max(ord_v, deg)
    # with open("Ord_v.csv", "a") as output: #Opens output file # Use the a parameter to add a row
    #     csvwriter = csv.writer(output, delimiter = ",")
    #     csvwriter.writerow((K, ord_v))
    return ord_v

def writhe(word):
    '''Returns the writhe of a braid word.
    author: Marc Kegel
    '''
    wr=0
    for w in word:
        wr=wr+sign(w)
    return wr

def cable(word,p,q):
    '''Returns a braid word of the cable. (For positive p and arbitrary q.)
    author: Marc Kegel
    '''
    cable_word=[]
    for i in word:
        subword=[]
        for t in range(0,p):
            subword=subword+list(range(p*abs(i)+t,p*(abs(i)-1)+t,-1))
        if i<0:
            subword=[-j for j in subword]
        cable_word=cable_word+subword
    wr=writhe(word)
    if (q-p*wr)<0:
        cable_word=cable_word+(p*wr-q)*list(range(-1,-(p-1)-1,-1))
    if (q-p*wr)>=0:
        cable_word=cable_word+(-p*wr+q)*list(range(1,(p-1)+1,+1))
    return cable_word

In [3]:
census = []
with open('L_space_status.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        census.append(str(row['knot']))

In [4]:
snappy.Manifold(census[1240])

o9_43443(0,0)

In [5]:
L_space = {'knot':'L_space'}
with open('L_space_status.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        L_space[str(row['knot'])] = eval(row['L-space knot'])

In [6]:
L_space[census[1240]]

False

In [7]:
genus_census = {'knot':'genus'}
with open('3genus.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        genus_census[str(row['knot'])] = eval(row['3-genus'])

In [8]:
genus_census[census[1240]]

8

In [29]:
'''
Checking Theorem 1.1 using census L-space knots
'''
with open("Theorem_census.csv", "a") as output: #Opens output file # Use the a parameter to add a row
    csvwriter = csv.writer(output, delimiter = ",")
    csvwriter.writerow(('knot', 'p', 'q', 'Ord', 'Ordv_pq', 'result'))
R.<t> = LaurentPolynomialRing(ZZ)
for p in range(2, 10):
    for k in census: 
        if not(L_space[k]):continue
        bnd = p*(2*genus_census[k] - 1)
        M = snappy.Manifold(k)
        A = M.alexander_polynomial()
        for i in range(1, 100):
            q = bnd + i
            if gcd(p,q) != 1: continue
            Ord_v = get_Ordv_laurent(R(A(t)))
            T_pq = R((t^(p*q) - 1)*(t - 1)/((t^p - 1)*(t^q - 1)))
            A_pq = T_pq(t) * R(A(t^p))
            Ordv_pq = get_Ordv_laurent(R(A_pq))
            quot = ((p*(Ord_v + 1)) == (Ordv_pq+1))
            #print(k, p, q, Ord_v, Ordv_pq, quot)
            with open("Theorem_census.csv", "a") as output: #Opens output file # Use the a parameter to add a row
                csvwriter = csv.writer(output, delimiter = ",")
                csvwriter.writerow((k, p, q, Ord_v, Ordv_pq, quot))


KeyboardInterrupt



In [35]:
with open("Theorem_HTLinkExteriors.csv", "a") as output: #Opens output file # Use the a parameter to add a row
    csvwriter = csv.writer(output, delimiter = ",")
    csvwriter.writerow(('knot', 'p', 'q', 'Ord', 'Ordv_pq', 'result'))
R.<t> = LaurentPolynomialRing(ZZ)
for p in range(2,10):
    for M in snappy.HTLinkExteriors(knots_vs_links='knots'): 
        K = M.link()
        HFK = K.knot_floer_homology()
        if (not(HFK['L_space_knot'])) or HFK['seifert_genus'] == 1: continue
        bnd = p*(2*HFK['seifert_genus'] - 1)
        A = M.alexander_polynomial()
        for i in range(1, 100):
            q = bnd + i
            if gcd(p,q) != 1: continue
            Ord_v = get_Ordv_laurent(R(A(t)))
            T_pq = R((t^(p*q) - 1)*(t - 1)/((t^p - 1)*(t^q - 1)))
            A_pq = T_pq(t) * R(A(t^p))
            Ordv_pq = get_Ordv_laurent(R(A_pq))
            quot = ((p*(Ord_v + 1)) == (Ordv_pq+1))
            #print(k, p, q, Ord_v, Ordv_pq, quot)
            with open("Theorem_HTLinkExteriors.csv", "a") as output: #Opens output file # Use the a parameter to add a row
                csvwriter = csv.writer(output, delimiter = ",")
                csvwriter.writerow((M.name(), p, q, Ord_v, Ordv_pq, quot))

In [34]:
'''
Checking Theorem 1.3 for trefoil
'''
with open("Theorem_T23.csv", "a") as output: #Opens output file # Use the a parameter to add a row
    csvwriter = csv.writer(output, delimiter = ",")
    csvwriter.writerow(('p', 'q', 'Ord', 'Ordv_pq', 'Result'))
R.<t> = LaurentPolynomialRing(ZZ)
M = snappy.Manifold('3_1')
A = M.alexander_polynomial()
for p in range(2, 10):
    for i in range(1, 100):
        q = p + i #p(2*g - 1) = p(2-1) = p
        if(gcd(p,q) != 1):continue
        Ord_v = get_Ordv_laurent(R(A(t)))
        T_pq = R(t^(p*q) - 1)*(t - 1)/((t^p - 1)*(t^q - 1))
        A_pq = T_pq(t) * A(t^p)
        Ordv_pq = get_Ordv_laurent(R(A_pq))
        quot = ((p*(Ord_v + 1)) == (Ordv_pq+1))
        quot_less = (p + q%p - 1) == (Ordv_pq)
        #print('3_1', p, q, Ord_v, Ordv_pq, quot)
        with open("Theorem_T23.csv", "a") as output: #Opens output file # Use the a parameter to add a row
            csvwriter = csv.writer(output, delimiter = ",")
            if q > 2*p: 
                csvwriter.writerow((p, q, Ord_v, Ordv_pq, quot))
            else: 
                csvwriter.writerow((p, q, Ord_v, Ordv_pq, quot_less))

In [43]:
with open("Question_mult_low_crossing.csv", "a") as output: #Opens output file # Use the a parameter to add a row
    csvwriter = csv.writer(output, delimiter = ",")
    csvwriter.writerow(('knot', 'p', 'q', 'Ord', 'Ordv_pq', 'Result'))

for p in range(2,5): 
    for M in snappy.HTLinkExteriors(knots_vs_links='knots'): 
        K = M.link()
        bw = K.braid_word()
        HFK = K.knot_floer_homology(complex=True)
        Ord = get_Ordv_from_complex(HFK)
        if HFK['L_space_knot']:continue
        g = HFK['seifert_genus']
        bnd = p*(2*g - 1)
        Ord = get_Ordv_from_complex(HFK)
        for i in range(1, 10):
            q = bnd + i
            if(gcd(p,q) != 1): continue
            K_pq = snappy.Link(braid_closure = cable(bw, p,q))
            K_pq.simplify('global')
            HFK_pq = K_pq.knot_floer_homology(complex=True)
            Ord_pq = get_Ordv_from_complex(HFK_pq)
            result = ((Ord_pq+1) / (Ord+1) == p)
            with open("Question_mult_low_crossing.csv", "a") as output: #Opens output file # Use the a parameter to add a row
                csvwriter = csv.writer(output, delimiter = ",")
                csvwriter.writerow((M.name(), p, q, Ord, Ord_pq, result))

KeyboardInterrupt: 

In [ ]:
with open("Question_mult_census.csv", "a") as output: #Opens output file # Use the a parameter to add a row
    csvwriter = csv.writer(output, delimiter = ",")
    csvwriter.writerow(('knot','p', 'q', 'Ord', 'Ordv_pq', 'Result'))

for p in range(2,5): 
    for w in words: 
        bw = w[1]
        K = snappy.Link(braid_closure = bw)
        K.simplify('global')
        HFK = K.knot_floer_homology(complex=True)
        Ord = get_Ordv_from_complex(HFK)
        if HFK['L_space_knot']:continue
        g = HFK['seifert_genus']
        bnd = p*(2*g - 1)
        Ord = get_Ordv_from_complex(HFK)
        for i in range(1, 10):
            q = bnd + i
            if(gcd(p,q) != 1): continue
            K_pq = snappy.Link(braid_closure = cable(bw, p,q))
            K_pq.simplify('global')
            HFK_pq = K_pq.knot_floer_homology(complex=True)
            Ord_pq = get_Ordv_from_complex(HFK_pq)
            result = ((Ord_pq+1) / (Ord+1) == p)
            # print(w[0], p, q, Ord, Ord_pq, result)
            with open("Question_mult_census.csv", "a") as output: #Opens output file # Use the a parameter to add a row
                csvwriter = csv.writer(output, delimiter = ",")
                csvwriter.writerow((w[0], p, q, Ord, Ord_pq, result))